# 2.1 - MLFlow

### Entranamiento del modelo y subida a MLFlow

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split as tts

from sklearn.linear_model import ElasticNet


import mlflow
import mlflow.sklearn

In [2]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [3]:
data=pd.read_csv('../data/winequality-red.csv', sep=';')

X=data.drop('quality', axis=1)

y=data.quality


X_train, X_test, y_train, y_test=tts(X, y)

In [4]:
X_test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1065,7.7,0.61,0.18,2.4,0.083,6.0,20.0,0.99630,3.29,0.60,10.2
1190,9.1,0.40,0.57,4.6,0.080,6.0,20.0,0.99652,3.28,0.57,12.5
1355,6.1,0.32,0.25,1.8,0.086,5.0,32.0,0.99464,3.36,0.44,10.1
151,9.2,0.52,1.00,3.4,0.610,32.0,69.0,0.99960,2.74,2.00,9.4
1003,6.8,0.36,0.32,1.8,0.067,4.0,8.0,0.99280,3.36,0.55,12.8


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
y.unique()

array([5, 6, 7, 4, 8, 3])

In [9]:
with mlflow.start_run():
    
    elastic=ElasticNet(alpha=0.5, l1_ratio=0.5)
    elastic.fit(X_train, y_train)

    y_pred=elastic.predict(X_test)

    rmse, mae, r2=eval_metrics(y_test, y_pred)

    print('  RMSE: ',  rmse)
    print('  MAE: ',  mae)
    print('  R2: ',  r2)

    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('r2', r2)
    mlflow.log_metric('mae', mae)

    #registro del modelo
    mlflow.sklearn.log_model(elastic, '/home/iudh/apuntes_clase/semana_8/mlruns/0/dc2f149d432e466f98707baa451d6e8f/artifacts/modelo')
    
    
    
    

  RMSE:  0.7173903089150576
  MAE:  0.5838329117595471
  R2:  0.14581102850582273


MlflowException: Invalid artifact path: '/home/iudh/apuntes_clase/semana_8/mlruns/0/dc2f149d432e466f98707baa451d6e8f/artifacts/modelo'. Names may be treated as files in certain cases, and must not resolve to other names when treated as such. This name would resolve to '/home/iudh/apuntes_clase/semana_8/mlruns/0/dc2f149d432e466f98707baa451d6e8f/artifacts/modelo'

### LLamada al servidor de MLFlow

puerto 5000

In [ ]:
!mlflow ui

### Nueva conexion para prediccion por terminal

Puerto 1234

In [ ]:
!mlflow models serve -m mlruns/0/fcc62cbb40c0497c8fe4660b000791d3/artifacts/modelo -p 1234

In [ ]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity",
"free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],
"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:1234/invocations


In [ ]:
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:1234/invocations

### Prediccion en Python

In [ ]:
modelo='file:///home/iudh/apuntes_clase/semana_8/mlruns/0/dc2f149d432e466f98707baa451d6e8f/artifacts/modelo'

modelo=mlflow.pyfunc.load_model(modelo)


modelo.predict(pd.DataFrame([[0, 1, 0.48, 0, 6.2, 29, 90, 1.2, 45, 75, 0.66]]))

In [ ]:
modelo.predict([[0, 1, 0.48, 0, 6.2, 29, 90, 1.2, 45, 75, 0.66]])